In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("storage-account", "adslproyecto01")

In [0]:
##Definicion de parametros
storage_account = dbutils.widgets.get("storage-account")
esquema_source = "raw"
esquema_destination = "bronze"
archivo_b3 = "dataset_rcau_B3.csv"

In [0]:
reporte_b3 = StructType(fields=[StructField("cod_ca_pa", StringType(), False),
                                     StructField("cod_c_arrib", StringType(), True),
                                     StructField("num_usu", StringType(), True),
                                     StructField("nomb_usu", StringType(), True),
                                     StructField("t_id", IntegerType(), True),
                                     StructField("num_id", StringType(), True),
                                     StructField("f_ini_at", StringType(), True),
                                     StructField("t_sol", IntegerType(), True),
                                     StructField("tramite", IntegerType(), True),
                                     StructField("servicio", StringType(), True),
                                     StructField("cod_atencion", StringType(), True),
                                     StructField("f_fin_at", StringType(), True),
                                     StructField("fecha_solicitado", StringType(), True),
                                     StructField("estado", StringType(), True),
                                     StructField("tiempo_espera", IntegerType(), True)
])

In [0]:
df_reporte_b3 = spark.read\
                    .option('header', True)\
                    .schema(reporte_b3)\
                    .csv(f"abfss://{esquema_source}@{storage_account}.dfs.core.windows.net/{archivo_b3}")


In [0]:
df_reporte_b3_select = df_reporte_b3.select(col("cod_ca_pa"), 
                                                col("cod_c_arrib"), 
                                                col("num_usu"), 
                                                col("nomb_usu"), 
                                                col("f_ini_at"),
                                                col("t_sol"), 
                                                col("tramite"),
                                                col("servicio"),
                                                col("fecha_solicitado"),
                                                col("estado"),
                                                col("tiempo_espera"))


In [0]:
df_reporte_b3_renamed = df_reporte_b3_select.withColumnRenamed("cod_ca_pa", "cod_oficina") \
.withColumnRenamed("cod_c_arrib", "cod_arribo") \
.withColumnRenamed("num_usu", "num_usuario") \
.withColumnRenamed("nomb_usu", "nomb_usuario") \
.withColumnRenamed("f_ini_at", "fec_inicio") \
.withColumnRenamed("t_sol", "tipo_solicitud")\
.withColumnRenamed("tramite", "tramite") \
.withColumnRenamed("servicio", "servicio") \
.withColumnRenamed("fecha_solicitado", "fec_solicitado") \
.withColumnRenamed("estado", "estado") \
.withColumnRenamed("tiempo_espera", "tiempo_espera")

In [0]:
 df_reporte_b3_final = df_reporte_b3_renamed.withColumn("fecha_registro", current_timestamp())

In [0]:
df_reporte_b3_final.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(f"abfss://{esquema_destination}@{storage_account}.dfs.core.windows.net/reporte_b3")

In [0]:
##Definicion de parametros
archivo_catalog = "catalogo_maestras.csv"

In [0]:
maestra_catalogo = StructType(fields=[StructField("codigo_catalogo", StringType(), False),
                                     StructField("id_catalogo", StringType(), True),
                                     StructField("descripcion_catalogo", StringType(), True),
                                     StructField("categorias", StringType(), True),
                                     StructField("codigo", IntegerType(), True),
                                     StructField("etiqueta", StringType(), True),
                                     StructField("estado", StringType(), True)
])

In [0]:
df_maestra_catalogo = spark.read\
                    .option('header', True)\
                    .schema(maestra_catalogo)\
                    .csv(f"abfss://{esquema_source}@{storage_account}.dfs.core.windows.net/{archivo_catalog}")

In [0]:
df_maestra_catalogo_select = df_maestra_catalogo.filter(col("codigo_catalogo").isin("M_3", "M_5"))

In [0]:
df_maestra_catalogo_final = df_maestra_catalogo_select.withColumn("fecha_registro", current_timestamp())

In [0]:
df_maestra_catalogo_final.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(f"abfss://{esquema_destination}@{storage_account}.dfs.core.windows.net/catalogo_maestras")